In [1]:
import pymongo, pandas as pd, bcrypt
from gerar_relatorios import gerar_relatorios
from mongo_url import mongo_url

In [2]:
myclient = pymongo.MongoClient(mongo_url)

# GERAR RELATÓRIOS DA INVESTIGAÇÃO

In [ ]:
#  EXEMPLO DE CHAMADA QUE GERA OS RELATÓRIOS PARA A INVESTIGAÇÃO x E OS SALVA NA RESPECTIVA PASTA.
# A PASTA DE DESTINO DEVE SER ONDE FICAM OS ARQUIVOS DO SERVIDOR WEB, 
# gerar_relatorios(id_inv, destination_path)

# C.R.U.D. USUÁRIOS

In [3]:
db_scdf_admin = myclient['SCDF_users']
db_user = db_scdf_admin['users']

In [4]:
def insert_user(user, passwd, hashed=True):
    if db_user.find_one({'user':user}):
        return False
    if hashed:
        senha = bcrypt.hashpw(passwd.encode('utf-8'), bcrypt.gensalt(10))
    else:
        senha = passwd
    db_user.insert_one({
        'user':user,
        'psswd':senha,
        'investigations':[]
    })

In [5]:
def update_user(old_user, new_user, passwd=True, hashed=True):
    if not db_user.find_one({'user':user}):
        return False
    if hashed:
        senha = bcrypt.hashpw(passwd.encode('utf-8'), bcrypt.gensalt(10))
    else:
        senha = passwd
    if passwd:
        db_user.update_one({'user':old_user},{'$set':{
            'user':old_user,
            'psswd':senha
        }})
    else:
        db_user.update_one({'user':old_user},{'$set':{
            'user':new_user
        }})

In [6]:
def update_user_inv(user, investigations):
    if not db_user.find_one({'user':user}):
        return False
    for inv in investigations:
        db_user.update_one({'user':user},{'$push':{
            'investigations':{
                'nome_investigacao':inv['nome_investigacao'],
                'id_inv':inv['id_inv']
            }
        }})

In [7]:
def delete_user(user):
    try:
        db_user.delete_one({'user':user})
        return True
    except:
        return False

In [8]:
def find_users():
    for u in db_user.find({}):
        print(u)

In [9]:
# DELETAR USUÁRIO
delete_user('danilo')

True

In [10]:
# INSERIR USUÁRIO
insert_user('danilo', '123', hashed=False)

In [11]:
# ATUALIZAR USUÁRIO JÁ EXISTENTE (SENHA OU NOME DE USUÁRIO)
# update_user(old_user, new_user, passwd=True, hashed=True)

In [12]:
# ADICIONAR INVESTIGAÇÕES PARA USUÁRIO EXISTENTE
# CADA INVESTIGAÇÃO É UM DICIONÁRIO: {'nome_investigacao':'Investigação...','id_inv':'...'}
update_user_inv('danilo', [{'nome_investigacao':'Teste 1','id_inv':'455b8232'}])

In [13]:
# LISTAR TODOS OS USUÁRIOS
find_users()

{'_id': ObjectId('5db721b5e44b7120b1c4438d'), 'user': 'danilo', 'psswd': '123', 'investigations': [{'nome_investigacao': 'Teste 1', 'id_inv': '455b8232'}]}


# DELETAR INVESTIGAÇÃO

In [3]:
id_inv = None

In [4]:
if id_inv:
    db_investigacao = myclient["SCDF_"+id_inv]

In [ ]:
def drop_inv(db_investigacao):
    db_investigacao.drop() 
    return True